<a href="https://colab.research.google.com/github/urieliram/statistical/blob/main/Tarea3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea 3
*Repeat the steps of the prostate cancer example in Section 3.2.1 using Python, first as a uni-variate problem using the book's data set and then as a multi-variate problem with data from your own project. Calculate also the p-values and the confidence intervals for the model's coefficients for the uni-variate version. Experiment, using libraries, also with subset selection.*

Los datos utilizados en este cuaderno están disponibles aquí: [datasets](https://drive.google.com/drive/folders/159GnBJQDxTY9oYqPBZzdNghyb4Gd9pDS?usp=sharing) 


In [22]:
import numpy as np
import pandas as pd
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import scipy.stats
import statsmodels.api as sm
import seaborn as sn

## Regresión líneal en cáncer de próstata
A continuación usaremos un modelo de regresión líneal para resolver el problema de próstata del libro [liga](https://link.springer.com/book/10.1007/978-0-387-84858-7).

Iniciamos cargando los datos de entrenamiento y almacenamos en $X$ y $Y$:

In [23]:
 df = pd.read_csv('prostata.csv') #https://github.com/maitbayev/the-elements-of-statistical-learning/blob/master/data/prostate/prostate.info

In [24]:
## Procesamos datos de entrenamiento 
df1 = df.loc[df.iloc[:,10].isin(['T'])] ## T = training set
df2 = df1['lpsa']
df1 = df1[['lcavol','lweight','age','lbph','svi','lcp','gleason','pgg45']]

Para estandarizar:
https://www.listendata.com/2017/04/how-to-standardize-variable-in-regression.html

In [25]:
df1.std(numeric_only=True) 
df1.mean(numeric_only=True)
df1 = df1-df1.mean(numeric_only=True)
df1=df1/df1.std(numeric_only=True) 
print(df1)
X  = df1.to_numpy() ## Predictors
Y  = df2.to_numpy() ## Outcome

      lcavol   lweight       age  ...       lcp   gleason     pgg45
0  -1.523680 -1.797414 -1.965590  ... -0.836769 -1.031712 -0.896487
1  -1.857204 -0.643057 -0.899238  ... -0.836769 -1.031712 -0.896487
2  -1.468157 -1.961526  1.233468  ... -0.836769  0.378996 -0.213934
3  -2.025981 -0.720349 -0.899238  ... -0.836769 -1.031712 -0.896487
4  -0.452342 -0.406493 -0.366061  ... -0.836769 -1.031712 -0.896487
..       ...       ...       ...  ...       ...       ...       ...
90  1.555621  0.998130  0.433703  ... -0.836769 -1.031712 -0.896487
91  0.981346  0.107969 -0.499355  ... -0.836769  0.378996 -0.384573
92  1.220657  0.525153  0.433703  ...  1.096538  0.378996  1.151171
93  2.017972  0.568193 -2.765355  ...  1.701433  0.378996  0.468618
95  1.262743  0.310118  0.433703  ...  1.265298  0.378996  1.833724

[67 rows x 8 columns]


Ahora, cargamos los datos de prueba en los arreglos $Xt$ y $Yt$:

In [26]:
## Procesamos datos de prueba 
dft = df.loc[df.iloc[:,10].isin(['F'])] ## F = test set
dft2 = dft['lpsa']
dft1 = dft[['lcavol','lweight','age','lbph','svi','lcp','gleason','pgg45']]
Xt  = dft1.to_numpy() ## Predictors
Yt  = dft2.to_numpy() ## Outcome

La matriz de correlación de los predictores

A continuación, obtenemos un modelo de predicción de los datos de entrenamiento usando regresión lineal. Posteriomente, calculamos los errores entre la predicción `y_pred` y los datos de entrenamiento $Y$. Los errores de la predicción con datos de entrenamiento son representados por un histograma.

In [32]:
df1 = sm.add_constant(X)
results = sm.OLS(endog = df2, exog = df1, hasconst = True).fit()

# Se ordenan los coeficientes por p-value
p_values = results.pvalues.sort_values(ascending = False)
print(results.pvalues)
print(results.params)
print(results.tvalues)
print(results.summary())
#if ('const' in p_values.index):
    # Se obtiene la constante
#    const = results.params.const
    # Se elimina la constante
#    p_values.drop('const', inplace = True)

    # [ ] Excepción de que todos sean mayores (se elimina todo)
    # [x]: Aumentar valor p a 0.1
    # [ ]: Usar todas :P
alpha = 0.05

const    1.537669e-35
x1       1.469415e-06
x2       7.917895e-03
x3       1.680626e-01
x4       4.430784e-02
x5       1.650539e-02
x6       6.697085e-02
x7       8.838923e-01
x8       8.754628e-02
dtype: float64
const    2.452345
x1       0.716407
x2       0.292642
x3      -0.142550
x4       0.212008
x5       0.309620
x6      -0.289006
x7      -0.020914
x8       0.277346
dtype: float64
const    28.181527
x1        5.366290
x2        2.750789
x3       -1.395909
x4        2.055846
x5        2.469255
x6       -1.866913
x7       -0.146681
x8        1.737840
dtype: float64
                            OLS Regression Results                            
Dep. Variable:                   lpsa   R-squared:                       0.694
Model:                            OLS   Adj. R-squared:                  0.652
Method:                 Least Squares   F-statistic:                     16.47
Date:                Sat, 22 Jan 2022   Prob (F-statistic):           2.04e-12
Time:                        

In [28]:
error = Y - y_pred
df = pd.DataFrame(error,Y) #df.to_csv('error.csv')
err_regress = mean_absolute_error(Y,y_pred)

NameError: ignored

In [ ]:
def dibuja_hist(df,colour,name,Xlabel,Ylabel,title):
    plt.figure()
    df.hist(column=0, bins=25, grid=False, figsize=(6,3), color=colour, zorder=2, rwidth=0.9)
    plt.xticks(rotation=90)
    plt.xlabel(Xlabel)
    plt.ylabel(Ylabel)
    #https://stackoverflow.com/questions/9662995/matplotlib-change-title-and-colorbar-text-and-tick-colors/42020486
    mytitle = plt.title(title) # get the title property handler   #plt.getp(title_obj)  
    plt.setp(mytitle, color='#ff8000')                            # set the color of title
    myax = plt.axes()   # get the axis property handler           # plt.getp(myax) print its propieties
    myax.xaxis.label.set_color('#ff8000')
    myax.yaxis.label.set_color('#ff8000')
    myax.tick_params(colors='#ff8000', which='both')              # myax.spines['bottom'].set_color('yellow')
    plt.savefig(name, transparent=True)
    plt.show() # o plt.save_en_algún_formato()

In [ ]:
dibuja_hist(df,colour='#17cb49',name='hist0.png',Xlabel="Error",Ylabel="Frecuencia",title="Errores de predicción en cáncer de próstata (entrenamiento)")
#df.hist(column=0, bins=25, grid=False, figsize=(6,3), color='#17cb49', zorder=2, rwidth=0.9)

Ahora, utilizamos el modelo obtenido con los datos de entrenamiento para predecir los datos de prueba. Además,  calculamos los errores entre la predicción `y_pred2` y los datos de prueba $Yt$. Los errores de la predicción con datos de prueba son representados por un histograma.

In [ ]:
y_pred2 = model.predict(Xt)
error2 = Yt - y_pred2
df = pd.DataFrame(error2,Yt) #df.to_csv('error2.csv')
err_regress_t = mean_absolute_error(Yt,y_pred2)

In [ ]:
dibuja_hist(df,colour='#17cb49',name='hist0.png',Xlabel="Error",Ylabel="Frecuencia",title="Errores de predicción en cáncer de próstata (prueba)")
#df.hist(column=0, bins=25, grid=False, figsize=(6,3), color='#17cb49', zorder=2, rwidth=0.9)

Por último calculamos el **error absoluto medio (MAE)** de los datos de entrenamiento así como de los datos de prueba.

In [ ]:
print("MAE del modelo de regresión con datos de entrenamiento:", err_regress)
print("MAE del modelo de regresión con datos de prueba:", err_regress_t) 

https://www.cienciadedatos.net/documentos/31_seleccion_de_predictores_subset_selection_ridge_lasso_dimension_reduction#Introducci%C3%B3n

https://gist.github.com/jseabold/5509329

In [ ]:
#https://towardsdatascience.com/a-comparison-of-shrinkage-and-selection-methods-for-linear-regression-ee4dd3a71f16